# Assignment 6: Urban Street Networks and Interactive Web Maps

**NOTE: THIS ASSIGNMENT IS OPTIONAL**. You must complete one out of homeworks #4, #5, and #6.

**Due date: Thursday, 11/14 by the end of class**

The notebook should be submitted to your own private repository on GitHub, which can be created using the following link:

https://classroom.github.com/a/OABZHpe3

#### Part 1: Visualizing crash data in Philadelphia

In this section, you will use `osmnx` to analyze the crash incidence in Center City. 

#### Part 2: Interactive web maps in Folium

In this section, you'll visualize a geospatial data set, queried using an API, using Folium in Python.

## Part 1: Visualizing crash data in Philadelphia

### 1.1 Load the geometry for the region being analyzed

We'll analyze crashes in the "Central" planning district in Philadelphia, a rough approximation for Center City. [Planning districts](https://www.opendataphilly.org/dataset/planning-districts) can be loaded from Open Data Philly. Read the data into a GeoDataFrame using the following link:

http://data.phl.opendata.arcgis.com/datasets/0960ea0f38f44146bb562f2b212075aa_0.geojson

Select the "Central" district and extract the geometry polygon for only this district. After this part, you should have a polygon variable of type `shapely.geometry.polygon.Polygon`.

### 1.2 Get the street network graph

Use OSMnx to create a network graph (of type 'drive') from your polygon boundary in 1.1.

### 1.3 Convert your network graph edges to a GeoDataFrame

Use OSMnx to create a GeoDataFrame of the network edges in the graph object from part 1.2. The GeoDataFrame should contain the edges but not the nodes from the network.

### 1.4 Load PennDOT crash data

Data for 2018 crashes (of all types) is available at the following path:

`./data/penndot_crashes/CRASH_2018_Philadelphia.csv`

The data was downloaded for Philadelphia County [from here](https://crashinfo.penndot.gov/PCIT/welcome.html).

### 1.5 Convert the crash data to a GeoDataFrame

You will need to use the `DEC_LAT` and `DEC_LONG` columns for latitude and longitude.

The full data dictionary for the data is [available here](http://pennshare.maps.arcgis.com/sharing/rest/content/items/ffe20c6c3c594389b275c6772a281bcd/data)

### 1.6 Trim the crash data to Center City

1. Get the boundary of the edges data frame (from part 1.3). Accessing the `.geometry.unary_union.convex_hull` property will give you a nice outer boundary region.
1. Trim the crashes using the `within()` function of the crash GeoDataFrame to find which crashes are within the boundary.

There should be about 1,600 crashes within the Central district.

### 1.7 Find the nearest edge for each crash

See: `ox.get_nearest_edges()`. It takes three arguments:

- the network graph
- the latitude of your crash data (the `y` attribute of the `geometry` column)
- the longitude of your crash data (the `x` attribute of the `geometry` column)

You will get a numpy array with 2 columns that represent `(u,v)` where each `u` and `v` are a node coordinates that identify unique nodes in the network.

**Note: this operation is slow and can take about 5 to 10 minutes, based on your computer speed.**

### 1.8 Calculate the total number of crashes per street

1. Make a DataFrame from your data from part 1.7 with two columns, `u` and `v`
1. Group by `u` and `v` and calculate the size
1. Reset the index and name your `size()` column as `crash_count`

After this step you should have a DataFrame with three columns: `u`, `v`, and `crash_count`.

### 1.9 Merge your edges GeoDataFrame and crash count DataFrame

You can use pandas to merge them on the `u` and `v` columns. This will associate the total crash count with each edge in the street network. 

**Tips:** 
   - Use a `left` merge where the first argument of the merge is the edges GeoDataFrame. This ensures no edges are removed during the merge.
   - Use the `fillna(0)` function to fill in missing crash count values with zero.

### 1.10 Calculate a "Crash Index"

Let's calculate a "crash index" that provides a normalized measure of the crash frequency per street. To do this, we'll need to:

1. Calculate the total crash count divided by the street length, using the `length` column
1. Perform a log transformation of the crash/length variable — use numpy's `log10()` function
1. Normalize the index from 0 to 1 (see the lecture notes for an example of this transformation)

**Note: since the crash index involves a log transformation, you should only calculate the index for streets where the crash count is greater than zero**.

After this step, you should have a new column in the data frame from 1.9 that includes a column called part 1.9.

### 1.11 Plot a histogram of the crash index values

Use matplotlib's `hist()` function to plot the crash index values from the previous step.

You should see that the index values are Gaussian-distributed, providing justification for why we log-transformed!

### 1.12 Plot the street networks, colored by the crash index

You can use GeoPandas to make the plot, coloring the streets by the crash index column.

**Tip:** if you use the viridis color map, try setting the facecolor of the axes as black for better constrast of the colors.

### 1.13 An interactive map of the crash index

In this part, we'll use Folium to make an interactive version of the map from the previous section. In this part, you will need to:

1. Initialize a Folium map centered on Philadelphia. The "Cartodb dark_matter" will be best if you want to use the viridis color map.
1. Add the street edges polygons to the map using the `Folium.GeoJson()` function.
1. Use a style function that applies a color to the edge geometries based on the value of the "crash index" column. 
    - See the crash index is defined from 0 to 1, you can pass this directly to a color map object, as in lecture.
    - You will need to convert the RGB color returned by the color map to a hex string
    - You can also set the "weight" attribute to change the width of the streets.
1. Add a GeoJsonTooltip object that includes the street name and crash index value so you can quickly identify which streets have the highest index values.

**Note:** if the Folium map is not rendering in the notebook, try removing unused columns — you should really only need the "geometry", "crash_index", and "name" columns.

## Part 2: Interactive web maps with Folium

In this part, you'll visualize a geospatial data set, queried using an API, using Folium in Python. The choice of data set is up to you, but must satisfy a few requirements:

- The data must be pulled using an API
- The data should be in GeoJSON format and be Point features (latitude, longitude)
- Use Folium to create a map showing a heat map of the data, using the Leaflet.heat plugin

#### Notes

- See the [week 9 lecture](https://github.com/MUSA-620-Fall-2019/week-9/blob/master/lecture-9.ipynb) for example maps using the Philadelphia shootings data set. **Note: you must choose a data set other than the shootings data set**.
- There are several options for data on OpenDataPhilly — any data set hosted on the CARTO SQL database with associated API documentation will work, similar to the shootings data set (you can use the `carto2gpd` library to do the querying).
- You can also choose a different API to use, as long as it satisfies the above requirements. To query the API, you can either use the `requests` module in Python or use `geopandas` directly to read the geojson returned by the API. For example:
  - the [Philadelphia bike share Indego](https://www.rideindego.com/about/data/) has an API of live station data in GeoJSON format: https://www.rideindego.com/stations/json/
  - Data.gov maintains a list of APIs in GeoJSON format: https://catalog.data.gov/dataset?res_format=GeoJSON. Note that not all of these are Point features
